In [9]:
# -------------- #

import pandas as pd
import os
import datetime
from datetime import date
import math

import load_data
# from featureEng import (
# )
# from models import grid_search
from config import (
    RAW_DATA_DIR,
    LAG_DICT,
    SLIDING_DICT
)
from models.baseline_avg_model import AvgModel
from sklearn.metrics import r2_score, mean_squared_error

import mlflow
import numpy as np
import mlflow.sklearn

# valid period. '2017-07-31' to '2017-08-16'
SPLIT_DATE = datetime.date(2017,7, 31)

def train_valid_split(df):
    valid_start = date(2017, 7, 26)
    valid_end = date(2017, 8, 10)
    
    valid_df = df[df['date']>=valid_start]
    valid_df = valid_df[valid_df['date']<valid_end]
    
    train_df = df[df['date']<valid_start]
    return train_df, valid_df

In [3]:
def add_weekday(df):
    df['weekday'] = df['date'].dt.weekday
    return df

In [8]:
train_df, test_df = load_data.main()
train_df = train_df.dropna()
train_df = train_df[train_df['date']>=datetime.date(2017, 5, 1)]


/Users/liuyu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  This is separate from the ipykernel package so we can avoid doing imports until


NameError: name 'date' is not defined

In [10]:
train_df, valid_df = train_valid_split(train_df)
train_df = add_weekday(train_df)
valid_df = add_weekday(valid_df)

/Users/liuyu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
/Users/liuyu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
/Users/liuyu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be ra

In [11]:
class AvgModel(object):
    def __init__(
                self, 
                label,
                train_days):
        self.label = label 
        self.train_days = train_days
        return

    def averaging_by_weekday(
                            self, 
                            df_item, 
                            column):
        """
        
        """
        result = {}
        for k in range(7):
            chunk = df_item[df_item['weekday'] == k]
            if self.train_days > 0:
                start_date = chunk['date'].iloc[-1] - \
                    pd.Timedelta(days=self.train_days)
                chunk = chunk[chunk['date'] >= start_date]
                # print(chunk)
            result[k] = chunk[column].mean()
        return result

    def fit(self, df):
        self.model = {}
        self.model_item = {}
        store_list = df['store_nbr'].unique()
        for store_id in store_list:
            df_store = df[df['store_nbr'] == store_id]
            item_list = df_store.item_nbr.unique()
            self.model[store_id] = {}
            for item_id in item_list:
                df_item = df_store[df_store['item_nbr'] == item_id]
                result =  self.averaging_by_weekday(df_item, self.label)
                self.model[store_id][item_id] = result
                if item_id not in self.model_item:
                    pass
                else:
                    o_r = self.model_item[item_id]
                    for k in range(7):
                        result[k] = (result[k]+o_r[k])/2
                self.model_item[item_id] = result
        return

    def predict(self, valid_df):
        def create_fn(row):
            try:
                return self.model[row['store_nbr']][row['item_nbr']][row['weekday']]
            except:
                try:
                    return self.model_item[row['item_nbr']][row['weekday']]
                except:
                    return 0
        pred = valid_df.apply(create_fn, axis=1)
        return pred

In [23]:
mlflow.set_experiment('grocery forecasting')
with mlflow.start_run(run_name='avg_model'):
    mlflow.log_param('model', 'avg_model')
#     avg_model = AvgModel(label='unit_sales',
#                         train_days=-1)
#     avg_model.fit(train_df)
#     _pred = avg_model.predict(valid_df)
    mse = mean_squared_error(valid_df['unit_sales'].values, _pred.fillna(0))
    mlflow.log_metric('mse', mse)

INFO: 'grocery forecasting' does not exist. Creating a new experiment
